### Loss Calulations for depth sensing

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
x = torch.rand(2, requires_grad=True)
x.backward(torch.zeros(2))
y = x.grad

In [7]:
########################## SSIM Loss ############################

from loss.loss_ssim import get_ssim_loss
img1 = torch.rand(1,1,22,22, requires_grad=True)
img2 = torch.rand(1,1,22,22, requires_grad = True)

print(get_ssim_loss(img1, img2))


SSIM of the inputs =======> 0.09407438337802887 with Loss as =======> 0.45296281576156616
tensor([0.4530], grad_fn=<DivBackward0>)


In [8]:
############################# Pointwise loss #####################
from loss.loss_point_wise import get_pointwise_loss

img1 = torch.rand(1,1,22,22, requires_grad=True)
img2 = torch.rand(1,1,22,22, requires_grad = True)
print(get_pointwise_loss(img1, img2))

Point Wise loss for GT and Pred images =======> 0.33211052417755127
tensor(0.3321, grad_fn=<MeanBackward0>)


In [9]:
############################ Gradient Loss ########################

######### Need to visit again for sum of grad along x and y axis ############
from loss.loss_gradient import get_grad_loss

img1 = torch.rand(1,1,22,22, requires_grad = True)
img2 = torch.rand(1,1,22,22, requires_grad = True)

img1.backward(torch.ones(1,1,22,22))
img2.backward(torch.ones(1,1,22,22))

print(get_grad_loss(img1, img2))

Loss gradient for the GT and Pred =======>2.0
tensor(2.)


In [10]:
import torch

from loss.loss import get_loss

img1 = torch.rand(1,1,22,22, requires_grad=True)
img2 = torch.rand(1,1,22,22, requires_grad = True)

img1.backward(torch.ones(1,1,22,22))
img2.backward(torch.ones(1,1,22,22))

print("Loss in Dense Depth is ==========> {}".format(get_loss(img1, img2).item()))

Point Wise loss for GT and Pred images =======> 0.3400869071483612
Loss gradient for the GT and Pred =======>2.0
SSIM of the inputs =======> -0.009599928744137287 with Loss as =======> 0.5047999620437622
Loss in Dense Depth is =======> 2.538808822631836
Loss in Dense Depth is ==========> 2.538808822631836


In [11]:
!pwd

/home/varinder/Documents/EVA4/S14-15/MYDenseDepth


In [2]:
from model import ResNet,Decoder
from torchsummary import summary
import torch

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
resnet18 = ResNet.ResNet18().to(device)
#decoder = Decoder.Decoder().to(device)
summary(resnet18, input_size=(3, 224, 224))
#summary(decoder, input_size=(512, 28, 28))

cpu
Decoder is called
Forward of Decoder is called with input shape is torch.Size([2, 512, 28, 28])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,728
       BatchNorm2d-2         [-1, 64, 224, 224]             128
            Conv2d-3         [-1, 64, 224, 224]          36,864
       BatchNorm2d-4         [-1, 64, 224, 224]             128
            Conv2d-5         [-1, 64, 224, 224]          36,864
       BatchNorm2d-6         [-1, 64, 224, 224]             128
        BasicBlock-7         [-1, 64, 224, 224]               0
            Conv2d-8         [-1, 64, 224, 224]          36,864
       BatchNorm2d-9         [-1, 64, 224, 224]             128
           Conv2d-10         [-1, 64, 224, 224]          36,864
      BatchNorm2d-11         [-1, 64, 224, 224]             128
       BasicBlock-12         [-1, 64, 224, 224]               0
   

### Train the network

In [ ]:
import train, test
import torch.optim as optim

optimizer = optim.SGD(mymodel.parameters(), lr=0.0001, momentum=0.9, weight_decay=0.0005)

EPOCHS = 10
for epoch in range(EPOCHS):  # loop over the dataset multiple times
    print("EPOCH: ", epoch)
    train.train(resnet18, device, trainloader, epoch, optimizer)
    test.test(resnet18, device, testloader, optimizer)

## Experiment with two images concatenated in pytorch

In [ ]:
from PIL import Image
import torchvision.transforms as transforms
import torch

img1 = Image.open('/home/varinder/Documents/EVA4/S14-15/Dataset/Dataset-S15/bg/bg1.jpg')
imgtensor1 = transforms.ToTensor()(img1).unsqueeze_(0)
print(imgtensor1.shape) 
img2 = Image.open('/home/varinder/Documents/EVA4/S14-15/Dataset/Dataset-S15/bg1/fg_1/overlay/1.jpg')
imgtensor2 = transforms.ToTensor()(img2).unsqueeze_(0)
print(imgtensor2.shape) 

imgtensor3 = torch.cat((imgtensor1,imgtensor2),1)
print(imgtensor3.shape)
img3 = transforms.ToPILImage()(imgtensor3.squeeze_(0))
print(img3.getbands())
img3.show()


## Testing how Upsampling works in pytorch

In [8]:
from PIL import Image
import torchvision.transforms as transforms
import torch
import torch.nn as nn

img1 = Image.open('/home/varinder/Documents/EVA4/S14-15/Dataset/Dataset-S15/bg1/fg_1/overlay/1.jpg')
imgtensor1 = transforms.ToTensor()(img1).unsqueeze_(0)
print(imgtensor1.shape) 

m = nn.Upsample(scale_factor=2, mode='bilinear')
img_upsampled = m(imgtensor1)
print(img_upsampled.shape)
img_up_pil = transforms.ToPILImage()(img_upsampled.squeeze_(0))
img_up_pil.show()

torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 448, 448])


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
img1 = Image.open('/home/varinder/Documents/EVA4/S14-15/Dataset/Dataset-S15/bg1/fg_1/overlay/1.jpg')
imgtensor1 = transforms.ToTensor()(img1).unsqueeze_(0)
print(imgtensor1.shape) 

img_upsampled = F.interpolate(imgtensor1, imgtensor1.shape[-1]*2 , mode='bilinear')
print(img_upsampled.shape)
img_up_pil = transforms.ToPILImage()(img_upsampled.squeeze_(0))
#img_up_pil.show()


torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 448, 448])
